In [1]:
"""
1. Json 등 형식에 맞는 파일 불러오기
2. article text를 추출
3. stop word 적용
4. text rank 핵심 문장으로 요약
5. //konply에서 명사만 추출 ex) okt.nouns(string)
6. 키워드 추출 (ex: TF-IDF)
7. 아니면 핵심 문장간 유사도 비교
8. 합당한 문장에서 지명 대조
9. 추출한 지명 따로 저장
"""

'\n1. Json 등 형식에 맞는 파일 불러오기\n2. article text를 추출\n3. stop word 적용\n4. text rank 핵심 문장으로 요약\n5. //konply에서 명사만 추출 ex) okt.nouns(string)\n6. 키워드 추출 (ex: TF-IDF)\n7. 아니면 핵심 문장간 유사도 비교\n8. 합당한 문장에서 지명 대조\n9. 추출한 지명 따로 저장\n'

In [113]:
# 1. csv file 불러오기
import pandas as pd
csv_data = pd.read_csv('201910article.csv', delimiter=',', encoding='UTF-8')

# 행렬 개수 확인
print("데이터 개수 :",csv_data.shape)
print(csv_data.shape[0])
print()

csv_data

데이터 개수 : (1526, 4)
1526



,date,popular,subject,article
0,20191001,society,前 오피스텔 성매매 업주 한 달 3000만원 벌어 손 뗄 수가 없었다,옆집이 성매매 오피스텔 중 형량은 낮고 돈은 많이 번다“5년간 오피 운영하다 적발 ...
1,20191001,society,조국 촛불에 깜짝 놀라檢개혁 역사적 대의 위해 모인 것,조국 법무부 장관이 1일 서울 여의도 국회 본회의장에서 열린 제...
2,20191001,society,단독최성해 정경심 압수수색 사흘전 서류 주지말라 전화,“학교에 검찰 압수수색 나오기 사흘 전쯤 정경심 교수가 나에게 전화해 ‘혹시 압수수...
3,20191001,society,윤석열 서울중앙지검 등 3개청 외 특수부 전면폐지개혁안 발표,윤석열 검찰총장 © 1 황기선 기자 서울뉴스1 ...
4,20191001,society,속도 붙은 태풍 미탁오늘부터 제주에 600 물폭탄,제17호 태풍 타파로 강풍이 몰아치는 강원도 안인진 해변 자료사...
...,...,...,...,...
1521,20191031,society,서울·경기서 하루 3번 성폭행 시도한 40대 배달업자 체포,서울 중랑경찰서 간판 연합뉴스 제공 서울연합뉴스...
1522,20191031,society,조국 내사 안 했다고 보기 어렵다 현직 검사 글 논란,앵커 유시민 노무현재단 이사장은 윤석열 검찰총장이 지난 8월 중순 한...
1523,20191031,society,헬기 아닌 배로 4시간 41분 만에참담한 구조 참사 당시,앵커 군을 헬기가 아닌 배로 병원에 옮기기까지의 순간순간은 특조위가 ...
1524,20191031,society,흉기 휘둘러 얼굴 다쳤는데사람 잘못 봤네요,뉴스데스크 ◀ 앵커 ▶아이들과 시민들이 몰려있는 대낮 도심 공원에서 30대 여성이 ...


In [144]:
article_list = csv_data.article
print(article_list[:10])

article_list[0].strip()

0    옆집이 성매매 오피스텔 중 형량은 낮고 돈은 많이 번다“5년간 오피 운영하다 적발 ...
1               조국 법무부 장관이 1일 서울 여의도 국회 본회의장에서 열린 제...
2    “학교에 검찰 압수수색 나오기 사흘 전쯤 정경심 교수가 나에게 전화해 ‘혹시 압수수...
3               윤석열 검찰총장 © 1 황기선 기자          서울뉴스1 ...
4               제17호 태풍 타파로 강풍이 몰아치는 강원도 안인진 해변 자료사...
5               【서울뉴시스】 전진환 기자  윤석열 검찰총장이 지난달 30일 오...
6    황교안  자유한국당 대표 투쟁은 문희상 의장 민주당 또 그 2중대 3중대의 불법적 ...
7               사진순천경찰서          전남 순천 한 종합병원에서 몰래카...
8    경향신문 ㆍ“대통령에 대한 압박이자 인사권에 정면 도전” 비판ㆍ“이 정도 됐으면 물...
9               서울 서초구 대검찰청 2019101뉴스1 © 1 구윤성 기자  ...
Name: article, dtype: object


'옆집이 성매매 오피스텔 중 형량은 낮고 돈은 많이 번다“5년간 오피 운영하다 적발 3개월만 범죄 인정… 추징금 전두환도 안 내700만원 받고 성매매 알선 컨설팅… 마사지업소 할 땐 月700만원 경찰 상납”           경찰이 성매매 오피스텔 단속 현장에서 촬영한 화장실 수건 등의 모습 화장실의 조명이 붉은 색인 것이 눈에 띈다 이혜미 기자          “오피성매매 오피스텔 할 때 한 달에 3000만원순익기준 정도 벌었죠 하루에 90만100만원 정도요 하루 일 끝나면 애들피고용인들이 집에 찾아와서 냉장고에다가 현금을 넣어놓고 갔어요 이 업포주을 하는 사람 중에 스스로 그만두는 사람은 없어요 10명 중 한 명 정도라고 해야 하나 단속 걸려서 어쩔 수 없이 그만 두는 경우죠”2010년부터 성매매 알선업을 해온 40대 이필성가명씨 그의 사건을 맡았었던 법조계 인사의 소개로 어렵게 만날 수 있었다 그는 한 달에 최고 7000만원을 벌었던 적도 있다고 말했다 이씨는 수 차례 단속에 걸렸지만 수익이 막대한 성매매 업주를 언제든 다시 하고 싶다는 생각을 숨기지 않았다 형량은 턱없이 낮고 추징은 미미하며 그만큼 수익은 높기 때문이다 성매매가 여전히 성업을 이루는 근본 원인을 확인할 수 있는 대목이다 그는 총 56차례 단속에 걸렸다고 한다 스스로 다 기억하지 못할 정도다 몇 번 적발됐는지 묻자 손가락을 꼽으며 기억을 더듬었다 “처음 걸렸을 때 벌금 150만원 그 다음에 벌금 350만원 또 200만원…” 징역형을 실제로 산 것은 마지막 적발됐을 때 단 1년 그 이전에는 세차례 벌금 이어 집행유예였다 “립카페 건마건전마사지를 하다가 오피를 45년 했어요 오피 여러 개를 운영했죠 오피로 잡혔을 때 3개월 했다고 진술했고 3개월만 인정 받았죠 그 기간만 했다고 진술하면 경찰도 방법이 없죠” 모두 현금 거래이고 현장 적발이 아니면 성매매는 입증이 어렵기 때문이다 “경찰이 집에 찾아와서 체포해서 갈 때 압수수색도 안 해요 어차피 현금 발견해봐야 증명 안되고 ‘제 돈인데요’ 하면

In [150]:
# 3_1. 특수문자 제거

# 소문자 변환
# re 라이브러리로 특수문자 제거

# 읽어온 데이터 확인 (json_body)


sentence = []

# 특수문자 제거 라이브러리
import re
for a in range(len(article_list)):
    sentence.append(article_list[a].strip())
    
for i in range(len(sentence)):
    sentence[i] = sentence[i].lower() # lower() : 모든 문자를 소문자로 변환
    sentence[i] = re.sub("(<br\s*/><br\s*/>)|(\-)|(\/)", ' ', sentence[i]) # 특수문자 제거
    sentence[i] = re.sub("[.;:!\'?,\"()|[\]]", ' ', sentence[i]) # 특수문자 제거

print("특수문자 제거 sentence[1]")
print(sentence[1])
print()

특수문자 제거 sentence[1]
조국 법무부 장관이 1일 서울 여의도 국회 본회의장에서 열린 제371회국회정기회 제5차 본회의에서 교육·사회·문화 분야 대정부 질의에 답변하고 있다 2019101뉴스1 © 1 임세영 기자          서울뉴스1 윤다정 기자전형민 기자  조국 수호와 검찰개혁을 주장하는 시민들이 서초동에서 대규모 촛불집회를 연 가운데 조국 법무부 장관은 깜짝 놀랐다며 검찰개혁이라는 시대적 과제 역사적 대의를 위해 모인 게 아닌가 생각한다고 말했다조 장관은 1일 국회 대정부질문에 참석해 촛불 시민들이 저 개인을 위해 나섰다고는 생각하지 않는다며 이같이 밝혔다그러면서 저의 부족함이나 불찰 때문에 시민들께서 많은 실망감을 가지셨을텐데 국민들께서 저를 꾸짖으시면서도 서초동에 모여 촛불을 드셨다는 소회도 함께 언급했다조 장관은 전날30일 정부과천청사에서 열린 제2기 법무·검찰 개혁위원회 발족식에 참석한 자리에서도 국민들의 검찰개혁에 대한 열망은 헌정 역사상 가장 뜨겁다며 지난 토요일 수많은 국민이 검찰개혁을 요구하며 광장에 모여 촛불을 들었다고도 언급했었다또한 검찰 권력은 강력한 힘을 갖고 있으면서도 제도적 통제 장치를 갖고 있지 않다며 누구도 함부로 되돌릴 수 없는 검찰개혁 방안을 국민의 눈높이에서 마련하고 실행할 수 있도록 힘을 모아 주시라고 요청했다1



In [151]:
# 3_2. 외부 Stop Word 리스트 불러오기 (stop word ranking 기반) 

# 불용어 처리 - stop_word 리스트 불러오기 & 전처리
stop_words = []
f = open("C:\Python\source\stop_word.txt", 'r', encoding='UTF-8')
while True:
    line = f.readline()
    if not line: break
    #print(line) #정상 출력 확인
    stop_words.append(line.rstrip('\n')) # 개행문자 제거 후 추가
f.close()

# 불러온 stop word 리스트 출력
#print(stop_words) 

In [157]:
# 3_3. 불러온 Stop Word 적용  
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

result = [] 
for w in sentence: 
    if w not in stop_words: 
        result.append(w) 
# 위의 4줄은 아래의 한 줄로 대체 가능
# result=[word for word in word_tokens if not word in stop_words]

# 불용어 처리 결과 출력
print("stop word 필터링 결과 (result)")
print(result[:2])
print()

stop word 필터링 결과 (result)
['옆집이 성매매 오피스텔 중 형량은 낮고 돈은 많이 번다“5년간 오피 운영하다 적발 3개월만 범죄 인정… 추징금 전두환도 안 내700만원 받고 성매매 알선 컨설팅… 마사지업소 할 땐 月700만원 경찰 상납”           경찰이 성매매 오피스텔 단속 현장에서 촬영한 화장실 수건 등의 모습 화장실의 조명이 붉은 색인 것이 눈에 띈다 이혜미 기자          “오피성매매 오피스텔 할 때 한 달에 3000만원순익기준 정도 벌었죠 하루에 90만100만원 정도요 하루 일 끝나면 애들피고용인들이 집에 찾아와서 냉장고에다가 현금을 넣어놓고 갔어요 이 업포주을 하는 사람 중에 스스로 그만두는 사람은 없어요 10명 중 한 명 정도라고 해야 하나 단속 걸려서 어쩔 수 없이 그만 두는 경우죠”2010년부터 성매매 알선업을 해온 40대 이필성가명씨 그의 사건을 맡았었던 법조계 인사의 소개로 어렵게 만날 수 있었다 그는 한 달에 최고 7000만원을 벌었던 적도 있다고 말했다 이씨는 수 차례 단속에 걸렸지만 수익이 막대한 성매매 업주를 언제든 다시 하고 싶다는 생각을 숨기지 않았다 형량은 턱없이 낮고 추징은 미미하며 그만큼 수익은 높기 때문이다 성매매가 여전히 성업을 이루는 근본 원인을 확인할 수 있는 대목이다 그는 총 56차례 단속에 걸렸다고 한다 스스로 다 기억하지 못할 정도다 몇 번 적발됐는지 묻자 손가락을 꼽으며 기억을 더듬었다 “처음 걸렸을 때 벌금 150만원 그 다음에 벌금 350만원 또 200만원…” 징역형을 실제로 산 것은 마지막 적발됐을 때 단 1년 그 이전에는 세차례 벌금 이어 집행유예였다 “립카페 건마건전마사지를 하다가 오피를 45년 했어요 오피 여러 개를 운영했죠 오피로 잡혔을 때 3개월 했다고 진술했고 3개월만 인정 받았죠 그 기간만 했다고 진술하면 경찰도 방법이 없죠” 모두 현금 거래이고 현장 적발이 아니면 성매매는 입증이 어렵기 때문이다 “경찰이 집에 찾아와서 체포해서 갈 때 압수수색도 안 해요 어차피 

In [161]:
# 4. Text Rank로 문장 요약하기
# 참고 링크 : https://github.com/wnsgml972/TextRank-Doc-Summary

# 5. Konlpy 추가, Okt 사용, 명사만 추출 (전처리 과정)
from konlpy.tag import Okt
okt = Okt()
nouns = []
noun_article_list = []
# 명사 추출에 집어넣기 좋게 텍스트로 변환
#strr = ''.join(result)

 
# sentence[]별 명사만 추출하기 (텍스트만 받음)
for i in result:
    strr = ''.join(result[i])
    nouns = okt.nouns(strr)
    for j in nouns:
        noun_article_list[i].append(nouns[j])

print("noun_article_list[0]")
print(noun_article_list[0])

TypeError: list indices must be integers or slices, not str

In [162]:
# 6 TF-IDF 모델 생성 및 그래프 생성
#!pip install sklearn
#!pip install scikit-learn
#!pip install scipy
from sklearn.feature_extraction.text import TfidfVectorizer
X = result
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

print(tfidf_matrix.shape)

(1526, 124781)


In [163]:
# 6 TF-IDF 모델 (유사도 분석)
from sklearn.feature_extraction.text import TfidfVectorizer
 
mydoclist = ['find what you love', 'do what you love', "don't do what you hate"]
 
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(mydoclist)
 
document_distances = (tfidf_matrix * tfidf_matrix.T)
 
print ('유사도 분석을 위한 ' + str(document_distances.get_shape()[0]) + 'x' + str(document_distances.get_shape()[1]) + ' matrix를 만들었습니다.')
 
print(document_distances.toarray())

유사도 분석을 위한 3x3 matrix를 만들었습니다.
[[1.         0.62111188 0.2554899 ]
 [0.62111188 1.         0.51770886]
 [0.2554899  0.51770886 1.        ]]


In [17]:
# 6_2 TextRank 알고리즘 적용
class Rank(object):
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [18]:
# 6_3 TextRank Class 구현
class TextRank(object):
    def __init__(self, text):
        
        print("text")
        print(text)
        self.sent_tokenize = SentenceTokenizer()
        
        self.sentences = self.sent_tokenize.text2sentences(text)
        print("sentences")
        print(self.sentences)
        self.nouns = self.sent_tokenize.get_nouns(text)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()

        for idx in index:
            summary.append(self.sentences[idx])
        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords



    

In [26]:
print(json_body)
tr = TextRank(json_body)

[일요서울｜파주 강동기 기자] 파주시는 아프리카돼지열병으로 인해 다수의 가을 축제와 행사가 취소돼 지역경제가 유례없는 침체기를 겪는 상황에서 소상공인들의 경영난 극복을 위해 다양한 지원책을 이끌어 내고 있다.15일 파주시는 대규모 점포인 ㈜신세계사이먼과 협약 체결을 통해 소상공인들을 위한 1억 원의 특례보증 출연금을 이끌어내어 대규모 점포와 지역 내 중소 상인과의 상생발전을 위한 토대를 마련했다. 협약과 동시에 담보능력이 없어 창업에 어려움을 겪거나 경영난을 겪고 있는 소상공인들이 1억 원의 10배수인 10억 원(50여 명)의 대출
text
[일요서울｜파주 강동기 기자] 파주시는 아프리카돼지열병으로 인해 다수의 가을 축제와 행사가 취소돼 지역경제가 유례없는 침체기를 겪는 상황에서 소상공인들의 경영난 극복을 위해 다양한 지원책을 이끌어 내고 있다.15일 파주시는 대규모 점포인 ㈜신세계사이먼과 협약 체결을 통해 소상공인들을 위한 1억 원의 특례보증 출연금을 이끌어내어 대규모 점포와 지역 내 중소 상인과의 상생발전을 위한 토대를 마련했다. 협약과 동시에 담보능력이 없어 창업에 어려움을 겪거나 경영난을 겪고 있는 소상공인들이 1억 원의 10배수인 10억 원(50여 명)의 대출
sentences
['[ 일요 서울｜ 파주 강동기 기자] 파주시는 아프리카 돼지 열병으로 인해 다수의 가을 축제와 행사가 취소돼 지역경제가 유례없는 침체기를 겪는 상황에서 소 상공인들의 경영난 극복을 위해 다양한 지원책을 이끌어 내고 있다.15', '일 파주시는 대규모 점포인 ㈜ 신세계 사이먼과 협약 체결을 통해 소 상공인들을 위한 1억 원의 특례보증 출연금을 이끌어 내 어 대규모 점포와 지역 내 중소 상인과의 상생발전을 위한 토대를 마련했다.', '협약과 동시에 담보능력이 없어 창업에 어려움을 겪거나 경영난을 겪고 있는 소 상공인들이 1억 원의 10 배 수인 10억 원 (50 여 명) 의 대출']


ValueError: empty vocabulary; perhaps the documents only contain stop words